1. Engineer your features. Here you do not have them for free. You need to think of possible ways for transforming the collected data into meaningful features. For some ideas, consider traditional features such as texture features, color features, bags of visual words or more powerful ones involving CNNs. If you cannot think of anything, talk to the professor for some ideas.

2. Propose classification techniques to solve the problem. Suggestions here are the CNN directly, or SVMs/Random Forests allied with CNNs through the use of transfer learning.

3. Consider using data augmentation in the training (what about in the testing as well?)

4. Observation: You are free to use any solution to help you extract the features at this point.

5. Report all of your results for the validation and test data. The labels for the test will be released one week before the deadline.